In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import arange
from pandas import read_table, set_option
from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, cross_validate, StratifiedKFold, ParameterGrid
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cluster import KMeans, Birch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, silhouette_score, make_scorer, r2_score, mean_squared_error, euclidean_distances
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from nltk import word_tokenize
from nltk.stem.snowball import EnglishStemmer

import graphviz
import dmba
%matplotlib inline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from dmba import classificationSummary, printTermDocumentMatrix, classificationSummary, liftChart

import plotly.express as px
import plotly.io as pio

In [4]:
orders = pd.read_excel('full_orders.xlsx')

In [26]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28821 entries, 0 to 28820
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     28821 non-null  object        
 1   email              28821 non-null  object        
 2   transaction_date   28821 non-null  datetime64[ns]
 3   time_spent_min     28821 non-null  int64         
 4   total_bill_amount  28821 non-null  float64       
 5   item_type          28821 non-null  object        
 6   item_id            28821 non-null  object        
 7   item_name          28821 non-null  object        
 8   category           28821 non-null  object        
 9   unit_price_cad     28821 non-null  float64       
 10  quantity           28821 non-null  int64         
 11  line_amount_cad    28821 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 2.6+ MB


In [25]:
orders['transaction_date'] = pd.to_datetime(orders['transaction_date'])

In [27]:
# For transaction-level metrics
transaction_summary = orders.groupby('transaction_id').agg({
    'email': 'first',
    'transaction_date': 'first',
    'time_spent_min': 'first',
    'total_bill_amount': 'first',  # or 'max' - they're all the same per transaction
    'line_amount_cad': 'sum'  # Sum items to verify total
}).set_index('transaction_date').resample('M')['total_bill_amount'].sum().reset_index()
# use transaction_summary for revenue analysis
total_revenue = transaction_summary['total_bill_amount'].sum()

# For customer-level analysis
customer_metrics = transaction_summary.groupby('email').agg({
    'transaction_id': 'count',  # Visit count
    'total_bill_amount': ['sum', 'mean'],  # Total and avg spend
    'time_spent_min': 'mean'
}).reset_index()

C:\Users\ztlu\AppData\Local\Temp\ipykernel_9276\4202257201.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  }).set_index('transaction_date').resample('M')['total_bill_amount'].sum().reset_index()


KeyError: 'email'

In [28]:
transaction_summary

,transaction_date,total_bill_amount
0,2024-10-31,3717.11
1,2024-11-30,27726.91
2,2024-12-31,40813.39
3,2025-01-31,22364.47
4,2025-02-28,22661.83
5,2025-03-31,29797.55
6,2025-04-30,28920.92
7,2025-05-31,30763.78
8,2025-06-30,35950.23
9,2025-07-31,38477.04


In [23]:
items = orders[['transaction_id', 'item_id', 'item_name', 'category', 
                'unit_price_cad', 'quantity', 'line_amount_cad']].copy()